In [1]:
import pandas as pd
import numpy as np

In [ ]:
from harmonic_inference.data.corpus_reading import aggregate_annotation_dfs
from pathlib import Path

ANNOTATIONS_PATH = Path('../corpora/annotations')
OUT_DIR = Path('corpus_data')

aggregate_annotation_dfs(ANNOTATIONS_PATH, OUT_DIR)

In [39]:
from harmonic_inference.data.corpus_reading import read_dump
from pathlib import Path

files_df = read_dump(Path('corpus_data', 'files.tsv'), index_col=0)
measures_df = read_dump(Path('corpus_data', 'measures.tsv'))
chords_df = read_dump(Path('corpus_data', 'chords.tsv'))
notes_df = read_dump(Path('corpus_data', 'notes.tsv'))

In [47]:
from harmonic_inference.utils import corpus_utils as cu
import importlib
importlib.reload(cu)

# Remove measure repeats
if isinstance(measures_df.iloc[0].next, tuple):
    measures_df = cu.remove_repeats(measures_df)

# Add offsets
if not all([column in notes_df.columns for column in ['offset_beat', 'offset_mc']]):
    notes_df = cu.add_note_offsets(notes_df, measures_df)

# Merge ties
#notes_df = cu.merge_ties(notes_df, measures=measures_df)

# Add chord metrical info
#chords_df = cu.add_chord_metrical_data(chords_df, measures_df)

In [48]:
files_df

,corpus_name,file_name
0,Bach-Suites,BWV806_01_Prélude.tsv
1,Bach-Suites,BWV806_02_Allemande.tsv
2,Bach-Suites,BWV806_03_Courante_I.tsv
3,Bach-Suites,BWV806_04_Courante_II.tsv
4,Bach-Suites,BWV806_05_Double_I.tsv
...,...,...
744,WFBach-Sonatas,Bach_WF_F02_Sonate_C-Dur.tsv
745,WFBach-Sonatas,Bach_WF_F03_Sonate_D-Dur.tsv
746,WFBach-Sonatas,Bach_WF_F03_Sonate_D-Dur_01.tsv
747,WFBach-Sonatas,Bach_WF_F03_Sonate_D-Dur_02.tsv


In [49]:
chords_df.loc[(chords_df.numeral == '@none') | chords_df.numeral.isnull()]

mc   mn  onset          label alt_label globalkey localkey  \
file_id chord_id                                                                
90      158        51   50      0  empty_harmony       NaN         F        I   
93      119        54   54      0  empty_harmony       NaN         C        V   
        217       109  108   9/16              ]       NaN         C       II   
        367       178  177      0           V[\\       NaN         C        i   
        369       178  177  15/16             /V       NaN         C        i   
...               ...  ...    ...            ...       ...       ...      ...   
748     430       110  110    1/6          V7/IV       NaN       NaN      NaN   
        431       110  110    1/4             IV       NaN       NaN      NaN   
        432       110  110    1/3             ii       NaN       NaN      NaN   
        433       110  110   5/12              V       NaN       NaN      NaN   
        434       110  110    1/2           I]\\       NaN       NaN      NaN   

                 pedal chord numeral  ... phraseend chord_type  \
file_id chord_id                      ...                        
90      158        NaN   NaN     NaN  ...       NaN        NaN   
93      119        NaN   NaN     NaN  ...       NaN        NaN   
        217        VII   NaN     NaN  ...       NaN        NaN   
        367          V   NaN     NaN  ...       NaN        NaN   
        369          V   NaN     NaN  ...       NaN        NaN   
...                ...   ...     ...  ...       ...        ...   
748     430        NaN   NaN     NaN  ...       NaN        NaN   
        431        NaN   NaN     NaN  ...       NaN        NaN   
        432        NaN   NaN     NaN  ...       NaN        NaN   
        433        NaN   NaN     NaN  ...       NaN        NaN   
        434        NaN   NaN     NaN  ...       NaN        NaN   

                 globalkey_is_minor localkey_is_minor chord_tones added_tones  \
file_id chord_id                                                                
90      158                   False             False          ()          ()   
93      119                   False             False          ()          ()   
        217                   False             False          ()          ()   
        367                   False              True          ()          ()   
        369                   False              True          ()          ()   
...                             ...               ...         ...         ...   
748     430                                                    ()          ()   
        431                                                    ()          ()   
        432                                                    ()          ()   
        433                                                    ()          ()   
        434                                                    ()          ()   

                  root bass_note volta special  
file_id chord_id                                
90      158       <NA>      <NA>  <NA>     NaN  
93      119       <NA>      <NA>  <NA>     NaN  
        217       <NA>      <NA>  <NA>     NaN  
        367       <NA>      <NA>  <NA>     NaN  
        369       <NA>      <NA>  <NA>     NaN  
...                ...       ...   ...     ...  
748     430       <NA>      <NA>  <NA>     NaN  
        431       <NA>      <NA>  <NA>     NaN  
        432       <NA>      <NA>  <NA>     NaN  
        433       <NA>      <NA>  <NA>     NaN  
        434       <NA>      <NA>  <NA>     NaN  

[2122 rows x 24 columns]

In [19]:
chords_df.loc[748].loc[chords_df.loc[748].mc_next == 45]

AttributeError: 'DataFrame' object has no attribute 'mc_next'

In [50]:
measures_df.loc[748].loc[measures_df.loc[748].mc.isin([43, 44, 45])]

,mc,mn,keysig,timesig,act_dur,offset,repeats,volta,barline,numbering_offset,dont_count,next
measure_id,,,,,,,,,,,,
42,43,43,2,2/2,3/4,0,endRepeat,<NA>,NaN,<NA>,<NA>,44
43,44,44,2,2/2,3/8,3/4,startRepeat,<NA>,start-repeat,<NA>,<NA>,45
44,45,45,2,2/2,1,0,NaN,<NA>,NaN,<NA>,<NA>,46


In [51]:
chords_df.loc[~chords_df.relativeroot.isnull() & chords_df.relativeroot.str.contains('/')].loc[:, ['label', 'chord', 'numeral','relativeroot']]

label         chord numeral relativeroot
file_id chord_id                                                 
93      307         viio6/V/IV    viio6/V/IV     vii         V/IV
        314          viio/V/II     viio/V/II     vii         V/II
97      459            .Fr6/iv        Fr6/iv       V         V/iv
        460           .Ger6/iv       Ger6/iv     vii         V/iv
        461            .It6/iv        It6/iv     vii         V/iv
...                        ...           ...     ...          ...
747     187           V6/V/III      V6/V/III       V        V/III
        188          V65/V/III     V65/V/III       V        V/III
        229       #viio7/V/III  #viio7/V/III    #vii        V/III
        230           V6/V/III      V6/V/III       V        V/III
        231          V65/V/III     V65/V/III       V        V/III

[269 rows x 4 columns]

In [52]:
measures_df.loc[1]

,mc,mn,keysig,timesig,act_dur,offset,repeats,volta,barline,numbering_offset,dont_count,next
measure_id,,,,,,,,,,,,
0,1,0,3,4/4,1/16,15/16,firstMeasure,<NA>,NaN,<NA>,1,2
1,2,1,3,4/4,1,0,NaN,<NA>,NaN,<NA>,<NA>,3
2,3,2,3,4/4,1,0,NaN,<NA>,NaN,<NA>,<NA>,4
3,4,3,3,4/4,1,0,NaN,<NA>,NaN,<NA>,<NA>,5
4,5,4,3,4/4,1,0,NaN,<NA>,NaN,<NA>,<NA>,6
5,6,5,3,4/4,1,0,NaN,<NA>,NaN,<NA>,<NA>,7
6,7,6,3,4/4,1,0,NaN,<NA>,NaN,<NA>,<NA>,8
7,8,7,3,4/4,1,0,NaN,<NA>,NaN,<NA>,<NA>,9
8,9,8,3,4/4,1,0,NaN,<NA>,NaN,<NA>,<NA>,10


In [53]:
notes_df.loc[1].loc[notes_df.loc[1].mc.isin([17, 18])]

,mc,mn,onset,duration,gracenote,nominal_duration,scalar,tied,tpc,midi,staff,voice,volta,offset_mc,offset_beat
note_id,,,,,,,,,,,,,,,
459,17,16,0,1/2,NaN,1/2,1,<NA>,4,52,2,2,<NA>,17,1/2
460,17,16,0,1/8,NaN,1/8,1,-1,3,57,2,1,<NA>,17,1/8
461,17,16,0,1/16,NaN,1/16,1,-1,5,59,1,2,<NA>,17,1/16
462,17,16,0,1/4,NaN,1/4,1,<NA>,4,64,1,1,<NA>,17,1/4
463,17,16,1/16,1/16,NaN,1/16,1,<NA>,5,59,1,2,<NA>,17,1/8
464,17,16,1/8,1/8,NaN,1/8,1,1,8,56,2,1,<NA>,17,1/4
465,17,16,1/8,1/16,NaN,1/16,1,<NA>,7,61,1,2,<NA>,17,3/16
466,17,16,3/16,1/16,NaN,1/16,1,<NA>,2,62,1,2,<NA>,17,1/4
467,17,16,1/4,1/16,NaN,1/16,1,-1,8,56,2,1,<NA>,17,5/16


In [9]:
import harmonic_inference.data.piece as piece
import harmonic_inference.utils.harmonic_utils as hu
from tqdm import tqdm


import logging
logging.basicConfig(filename='parsing.log', level=logging.INFO, filemode='w')

import importlib
importlib.reload(piece)
importlib.reload(hu)

for i in tqdm(files_df.index):
    logging.info(f"Parsing file id {i}")
    try:
        chords = chords_df.loc[i]
        notes = notes_df.loc[i]
        measures = measures_df.loc[i]
    except BaseException as e:
        logging.error(f"Error getting data for index {i}: {e}")
        continue

    try:
        score_piece = piece.ScorePiece(notes, chords, measures)
    except BaseException as e:
        logging.error(f"Error parsing index {i}: {e}")
        continue

100%|██████████| 749/749 [04:54<00:00,  2.54it/s]


In [ ]:
chords_df.loc[94].loc[chords_df.loc[94].mc == 71]

In [17]:
chords_df.loc[102]

,mc,mn,onset,label,alt_label,globalkey,localkey,pedal,chord,numeral,...,localkey_is_minor,chord_tones,added_tones,root,bass_note,volta,special,mc_next,onset_next,duration
chord_id,,,,,,,,,,,,,,,,,,,,,
0,1,0,3/8,.F.V,NaN,F,I,NaN,V,V,...,False,"(1, 5, 2)",(),1,1,<NA>,NaN,2,0,-1/4
1,2,1,0,I,NaN,F,I,NaN,I,I,...,False,"(0, 4, 1)",(),0,0,<NA>,NaN,2,3/8,3/8
2,2,1,3/8,V,NaN,F,I,NaN,V,V,...,False,"(1, 5, 2)",(),1,1,<NA>,NaN,3,0,1/8
3,3,2,0,I6,NaN,F,I,NaN,I6,I,...,False,"(4, 1, 0)",(),0,4,<NA>,NaN,3,3/8,3/8
4,3,2,3/8,I,NaN,F,I,NaN,I,I,...,False,"(0, 4, 1)",(),0,0,<NA>,NaN,4,0,1/8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,150,148,0,V7,NaN,F,I,NaN,V7,V,...,False,"(1, 5, 2, -1)",(),1,1,<NA>,NaN,150,1/8,1/8
305,150,148,1/8,V(64),NaN,F,I,NaN,V(64),V,...,False,"(1, 0, 4)",(),1,1,<NA>,NaN,150,1/4,1/8
306,150,148,1/4,ii64,NaN,F,I,NaN,ii64,ii,...,False,"(3, 2, -1)",(),2,3,<NA>,NaN,150,3/8,1/8


In [ ]:
import harmonic_inference_data as hid
import importlib
importlib.reload(hid)

datasets = {'global': {},
            'local': {},
            'none': {}}

datasets['global']['train'], datasets['global']['valid'], datasets['global']['test'] = hid.get_train_valid_test_splits(
    chords_df=chords_df, notes_df=notes_df, measures_df=measures_df, files_df=files_df,
    seed=0, h5_directory='data', h5_prefix='globalkey_811split', make_dfs=True, transpose_global=True
)

datasets['local']['train'], datasets['local']['valid'], datasets['local']['test'] = hid.get_train_valid_test_splits(
    chords_df=chords_df, notes_df=notes_df, measures_df=measures_df, files_df=files_df,
    seed=0, h5_directory='data', h5_prefix='localkey_811split', make_dfs=True, transpose_local=True
)

datasets['none']['train'], datasets['none']['valid'], datasets['none']['test'] = hid.get_train_valid_test_splits(
    chords_df=chords_df, notes_df=notes_df, measures_df=measures_df, files_df=files_df,
    seed=0, h5_directory='data', h5_prefix='811split', make_dfs=True
)

In [ ]:
import harmonic_inference_models as him
import harmonic_utils
import importlib
importlib.reload(him)
from ablation import get_masks_and_names
import torch
import os

masks, mask_names = get_masks_and_names()
dir_name = 'results'

data = 'local'
index = -1
prefix = (data + '_') if data != 'none' else ''

mask = torch.tensor(masks[index])
mask_name = mask_names[index]
checkpoint_dir = os.path.join(dir_name, prefix + mask_name)
log_file_name = os.path.join(dir_name, prefix + mask_name + '.log')

model = him.MusicScoreModel(len(datasets[data]['test'][0]['notes'][0]), len(harmonic_utils.CHORD_TYPES) * 12,
                            dropout=0.2, input_mask=mask)

print(mask_name)
print(checkpoint_dir)

In [ ]:
import os
import model_trainer
import importlib
importlib.reload(model_trainer)

from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR

optimizer = Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.001)
criterion = CrossEntropyLoss()
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10)
schedule_var = 'valid_loss'

trainer = model_trainer.ModelTrainer(model, train_dataset=datasets[data]['train'], valid_dataset=datasets[data]['valid'],
                                     test_dataset=datasets[data]['test'], seed=0, num_epochs=100, early_stopping=20,
                                     optimizer=optimizer, scheduler=scheduler, schedule_var=schedule_var,
                                     criterion=criterion,
                                     log_every=1, log_file_name=log_file_name,
                                     save_every=10, save_dir=checkpoint_dir, save_prefix='checkpoint',
                                     resume=os.path.join(checkpoint_dir, 'best.pth.tar'))

In [ ]:
trainer.train()

In [ ]:
trainer = model_trainer.ModelTrainer(model, train_dataset=datasets[data]['train'], valid_dataset=datasets[data]['valid'],
                                     test_dataset=datasets[data]['test'], seed=0, num_epochs=100, early_stopping=20,
                                     optimizer=optimizer, scheduler=scheduler, schedule_var=schedule_var,
                                     criterion=criterion,
                                     log_every=1, log_file_name=log_file_name,
                                     save_every=10, save_dir=checkpoint_dir, save_prefix='checkpoint',
                                     resume=os.path.join(checkpoint_dir, 'best.pth.tar'))

loss, acc, outputs, labels = trainer.evaluate(valid=False)
print(loss, acc)

In [ ]:
import eval_utils as eu
import harmonic_utils as hu
import matplotlib.pyplot as plt

label_strings = hu.get_one_hot_labels()
conf_mat = eu.get_conf_mat(labels, outputs)

plt.figure(figsize=(30,30))
plt.imshow(conf_mat, interpolation='none')
plt.colorbar()
plt.xticks(ticks=list(range(len(label_strings))), labels=label_strings, rotation=90, fontsize=10)
plt.yticks(ticks=list(range(len(label_strings))), labels=label_strings, fontsize=10)
plt.show()

In [ ]:
import eval_utils as eu

correct, incorrect = eu.get_correct_and_incorrect_indexes(labels, outputs)
print('Correct: ' + str(len(correct)))
print('Incorrect: ' + str(len(incorrect)))

In [ ]:
import eval_utils as eu
    
eu.print_result(incorrect[0], labels, outputs, limit=10, prob=False)

In [ ]:
import eval_utils as eu

chord, onset_notes, all_notes = eu.get_input_df_rows(incorrect[0], datasets[data]['test'])

print(chord)
print("USED NOTES:")
print(onset_notes)
print()
print("ALL NOTES:")
print(all_notes)

In [ ]:
import matplotlib.pyplot as plt
import eval_utils as eu

correct_ranks, indexes_by_rank = eu.get_correct_ranks(labels, outputs)
    
plt.figure(figsize=(30,30))
plt.bar(range(len(outputs[0])), [len(indexes) for indexes in indexes_by_rank])

In [ ]:
import eval_utils as eu
import importlib
importlib.reload(eu)

eval_df = eu.get_eval_df(labels, outputs, datasets[data]['test'])
eval_df

In [ ]:
import ablation
import importlib
importlib.reload(ablation)

dfs = ablation.load_all_ablated_dfs(directory='results', prefix=prefix[:-1] if len(prefix) > 0 else None)
_, mask_names = ablation.get_masks_and_names()

In [ ]:
import pandas as pd
import os

logs = []
for mask_name in mask_names:
    logs.append(pd.read_csv(os.path.join(os.path.join('results', prefix + mask_name + '.log'))))

In [ ]:
for df, log, mask_name in zip(dfs, logs, mask_names):
    print(f"{mask_name} Acc: {100 * df.correct.sum() / len(df)}")
    print(log.iloc[-1])

In [ ]:
import eval_utils as eu

global_df = eu.load_eval_df('results/global_no_ablation.csv')
local_df = eu.load_eval_df('results/local_no_ablation.csv')
none_df = eu.load_eval_df('results/no_ablation.csv')

In [ ]:
global_df

In [ ]:
global_counts = global_df.groupby(['correct_chord'])['correct'].agg(['mean', 'count']).sort_values('count', ascending=False)
local_counts = local_df.groupby(['correct_chord'])['correct'].agg(['mean', 'count']).sort_values('count', ascending=False)
none_counts = none_df.groupby(['correct_chord'])['correct'].agg(['mean', 'count']).sort_values('count', ascending=False)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,5))
plt.scatter(global_counts['count'], global_counts['mean'], color='red', label='Global key')
plt.scatter(local_counts['count'], local_counts['mean'], color='blue', label='Local key')
plt.scatter(none_counts['count'], none_counts['mean'], color='yellow', label='No transposition')
plt.title('Global key transposed')
plt.xlabel('Count')
plt.ylabel('Accuracy')
plt.legend()
plt.show()